# Testing DRPredICT Modules

**Authorship:**
Adam Klie, *03/29/2022*
***
**Description:**
Notebook for testing out modules for building DRPredICT architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    </ul>
</div>

In [18]:
import numpy as np
import pandas as pd
import torch
# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import sys
sys.path.append("../drpredict")

# Random Dataset

In [2]:
from torch.utils.data import dataset
from torch.utils.data import DataLoader

## VanillaAE

In [16]:
training_dataset = dataset.TensorDataset(torch.randn(1000, 1000))
training_dataloader = DataLoader(training_dataset, batch_size=32, num_workers=4)
training_dataset[0][0].shape

torch.Size([1000])

In [116]:
from VanillaAE import VanillaAE

In [117]:
encoder_args = {"activation": "sigmoid"}
decoder_args = {"dropout_rate": 0.2}

In [120]:
mutAE = VanillaAE(1000, 10, hidden_dims=[500, 100], encoder_kwargs=encoder_args, decoder_kwargs=decoder_args)

In [121]:
mutAE

VanillaAE(
  (encoder): FullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=1000, out_features=500, bias=True)
      (1): Sigmoid()
      (2): Linear(in_features=500, out_features=100, bias=True)
      (3): Sigmoid()
      (4): Linear(in_features=100, out_features=10, bias=True)
    )
  )
  (decoder): FullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=10, out_features=500, bias=True)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.2, inplace=False)
      (3): Linear(in_features=500, out_features=100, bias=True)
      (4): ReLU(inplace=True)
      (5): Dropout(p=0.2, inplace=False)
      (6): Linear(in_features=100, out_features=1000, bias=True)
    )
  )
)

In [29]:
for i_batch, batch in enumerate(training_dataloader):
    x = batch[0]
    outs = mutAE(x)
    print(x.shape, outs.shape)
    if i_batch==3:
        break

torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])
torch.Size([32, 1000]) torch.Size([32, 1000])


# Multiomic

In [95]:
training_dataset = dataset.TensorDataset(torch.randn(1000, 1000), torch.randn(1000, 1000), torch.randn(1000, 1000), torch.randn(1000))
training_dataloader = DataLoader(training_dataset, batch_size=32, num_workers=4)
training_dataset[0][0].shape, training_dataset[0][1].shape, training_dataset[0][2].shape, training_dataset[0][3]

(torch.Size([1000]), torch.Size([1000]), torch.Size([1000]), tensor(0.3439))

In [130]:
omic_types = ["mutation", "expression", "cn"]
in_dims = [1000, 1000, 1000]
out_dims = [6, 6, 6]
h_dims = [[500], [500], [500]]
encoder_args = [{"activation": "sigmoid"}, {"activation": "relu"}, {"activation": "sigmoid"}]
fcn_args = {"dropout_rate": 0.2, "output_dim": 1}

In [131]:
from MultiEncoder import MultiEncoder

In [132]:
drpredict = MultiEncoder(omics=omic_types, input_dims=in_dims, output_dims=out_dims, hidden_dims=h_dims, encoder_kwargs=encoder_args, fcn_kwargs=fcn_args)

In [128]:
for i_batch, batch in enumerate(training_dataloader):
    x = batch
    outs = drpredict(x)
    print(outs.shape)
    if i_batch==3:
        break

torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
